In [1]:
#| default_exp semantic_search_evaluator

%load_ext autoreload
%autoreload 2

In [2]:
#| export

import numpy as np
import pandas as pd
from typing import Dict, Set, List, Tuple
from transformers import AutoTokenizer, AutoModel
from enum import Enum
from torch import Tensor
import logging
import torch
import torch.nn.functional as F
from dotenv import load_dotenv
import os

load_dotenv()

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(message)s")


/Users/igor/workspace/mailio-ai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = os.getenv("MODEL_ID")
print(model_id)

intfloat/e5-small-v2


In [4]:
base_dir = "../data"
subfolder = model_id.split("/")[-1]
questions_file = f"{base_dir}/evaluation_dataset/sample_queries_cleaned.jsonl"

In [5]:
df_queries = pd.read_json(questions_file, lines=True)

In [6]:
df_queries.head(n=3)

,text,id,query
0,ARTIOM DRUMEA Sent You a Message Behance Basic...,<010001809eb828f6-b2888a42-bd74-4f16-b366-cf0e...,"""message from ARTIOM DRUMEA on Behance"""
1,Fwd: CIIS Commencement 2021: Join us on May 1!...,<BY5PR22MB208353712894331A58D4D4C6E72F9@BY5PR2...,"""CIIS Commencement 2021 details"""
2,Your Digest: From the Mac Startup Tone to the ...,<esuh8huNSw2sRLJpnIw4HQ@ismtpd0012p1iad2.sendg...,"""OneZero digital digest"""


In [7]:
embeddings_index_path = f"{base_dir}/{subfolder}/embeddings_index.npy"
embeddings_index = np.load(embeddings_index_path, allow_pickle=True)

In [8]:
# contruct the ground truth dictionary from the dataframe

ground_truth = dict[int, Set[int]]()
for idx, row in df_queries.iterrows():
    email_id = row["id"]
    # find in embeddings index it's index
    emb_index_set = set(np.where(embeddings_index == email_id)[0])
    ground_truth[idx] = emb_index_set

In [9]:
print(ground_truth[19])
# embeddings_index[2712]
print(np.where(embeddings_index == "<CPZP284MB058472D9136C52B8957CB8B192C69@CPZP284MB0584.BRAP284.PROD.OUTLOOK.COM>")[0])

{6391}
[1320 1321 1322]


In [10]:
#| export

class SimilarityFunction(Enum):
    COSINE = "cosine"
    DOT_PRODUCT = "dot"
    EUCLIDEAN = "euclidean"

class MailioInformationRetrievalEvaluator:
    """
    Insipired by on https://github.com/UKPLab/sentence-transformers/blob/v3.4-release/sentence_transformers/evaluation/InformationRetrievalEvaluator.py
    Gives me a bit more le-way to customize creation of query embeddings and corpus embeddings
    Also simplifies the code a bit

    Given a set of queries and a large corpus set. It will retrieve for each query the top-k most similar document. It measures
    Mean Reciprocal Rank (MRR), Recall@k, and Normalized Discounted Cumulative Gain (NDCG)
    """
    def __init__(self, 
        ground_truth:Dict[int, Set[int]],  # query_index => Set[corpus_index]
        corpus_embeddings: Tensor, # embeddingns of corpus_index
        query_embeddings: Tensor, # embeddings of query_index
        mrr_at_k: List[int] = [10],
        ndcg_at_k: List[int] = [10],
        accuracy_at_k: List[int] = [1, 3, 5, 10],
        precision_recall_at_k: List[int] = [1, 3, 5, 10],
        map_at_k: List[int] = [100],
        similarity_functions = [SimilarityFunction.COSINE],
        ) -> None:
        """
        Initializes the InformationRetrievalEvaluator.
        Args:
            corpus_embeddings (Tensor): A tensor of shape (N, D) containing the document embeddings.
            query_embeddings (Tensor): A tensor of shape (M, D) containing the query embeddings.
            ground_truth (Dict[str, Set[str]]): A dictionary mapping query index to a set of relevant document indexes.
            mrr_at_k (List[int]): A list of integers representing the values of k for MRR calculation. Defaults to [10].
            ndcg_at_k (List[int]): A list of integers representing the values of k for NDCG calculation. Defaults to [10].
            accuracy_at_k (List[int]): A list of integers representing the values of k for accuracy calculation. Defaults to [1, 3, 5, 10].
            precision_recall_at_k (List[int]): A list of integers representing the values of k for precision and recall calculation. Defaults to [1, 3, 5, 10].
            map_at_k (List[int]): A list of integers representing the values of k for MAP calculation. Defaults to [100].
        """
        self.corpus_embeddings = corpus_embeddings
        self.query_embeddings = query_embeddings
        self.ground_truth = ground_truth
        self.mrr_at_k = mrr_at_k
        self.ndcg_at_k = ndcg_at_k
        self.accuracy_at_k = accuracy_at_k
        self.precision_recall_at_k = precision_recall_at_k
        self.map_at_k = map_at_k
        self.similarity_functions = similarity_functions
        self.problematic_queries = set()

    def run(self):
        return self.compute_metrices()

    def get_problematic_queries(self):
        return self.problematic_queries

    def compute_metrices(self):
        """
        Computes the evaluation metrics.
        Args:
            top_k (int): The number of retrieved documents for which to compute the evaluation metrics. Defaults to 10.
        Returns:
            Dict[str, Dict[str, float]]: A dictionary mapping metric names to dictionaries mapping metric values to scores.
        """
        max_k = max(
            max(self.mrr_at_k),
            max(self.ndcg_at_k),
            max(self.accuracy_at_k),
            max(self.precision_recall_at_k),
            max(self.map_at_k),
        )
        # prepare the query result list for each query and each score function
        self.queries_result_list = {}
        
        metrics = {}
        for sim_fn in self.similarity_functions:
            similarity_name = str(sim_fn.value)
            queries_results = self.compute_similarity_function_product(sim_fn, top_k=max_k)
            similarity_metrics = self.compute_metrics(queries_results)
            metrics[similarity_name] = similarity_metrics
        return metrics
    
    def compute_similarity_function_product(self, similarity_function: SimilarityFunction, top_k: int = 100) -> Dict[int, List[Tuple[float, int]]]:
        """
        Computes the evaluation metrics for a given similarity function.
        Args:
            similarity_function (SimilarityFunction): The similarity function to use for computing the similarity between queries and documents.
            top_k (int): The number of retrieved documents for which to compute the evaluation metrics. Defaults to 10.
        Returns:
            Dict[int, List[Tuple[float, int]]] : A dictionary mapping query indexes to a list of tuples containing the similarity score and the document index.
        """

        query_results = {}
        
        # compute the similarity between each query and each document
        if similarity_function == SimilarityFunction.COSINE:
            for query_index in range(len(self.query_embeddings)):
                query_embedding = self.query_embeddings[query_index]
                similarity = F.cosine_similarity(query_embedding, self.corpus_embeddings, dim=1)
                scores, indices = similarity.topk(top_k, dim=0)
                s = scores.cpu().numpy().ravel()
                i = indices.cpu().numpy().ravel()
                query_results[query_index] = [(s, i) for s, i in zip(s, i)]
        
        if similarity_function == SimilarityFunction.DOT_PRODUCT:
            for query_index in range(len(self.query_embeddings)):
                query_embedding = self.query_embeddings[query_index]
                similarity = torch.matmul(self.corpus_embeddings, query_embedding)
                scores, indices = similarity.topk(top_k, dim=0)
                s = scores.cpu().numpy().ravel()
                i = indices.cpu().numpy().ravel()
                query_results[query_index] = [(s, i) for s, i in zip(s, i)]

        if similarity_function == SimilarityFunction.EUCLIDEAN:
            for query_index in range(len(self.query_embeddings)):
                query_embedding = self.query_embeddings[query_index]
                similarity = torch.cdist(query_embedding.unsqueeze(0), self.corpus_embeddings, p=2).squeeze()
                scores, indices = similarity.topk(top_k, dim=0, largest=False)
                s = scores.cpu().numpy().ravel()
                i = indices.cpu().numpy().ravel()
                query_results[query_index] = [(s, i) for s, i in zip(s, i)]

                
        return query_results    

    def compute_metrics(self, queries_results: Dict[int, List[Tuple[float, int]]]):
        """
        Computes the evaluation metrics for a given similarity function.
        Args:
            queries_results (Dict[int, List[Tuple[float, int]]]): A dictionary mapping query indexes to a list of tuples containing the similarity score and the document index.
        Returns:
            Dict[str, Dict[str, float]]: A dictionary mapping metric names to dictionaries mapping metric values to scores.
        """
        # Init score computation values
        num_hits_at_k = {k: 0 for k in self.accuracy_at_k}
        precisions_at_k = {k: [] for k in self.precision_recall_at_k}
        recall_at_k = {k: [] for k in self.precision_recall_at_k}
        MRR = {k: 0 for k in self.mrr_at_k}
        ndcg = {k: [] for k in self.ndcg_at_k}
        AveP_at_k = {k: [] for k in self.map_at_k}

        # queries not in top 3
        self.problematic_queries = set()

        # Compute scores on results
        for query_index, results in queries_results.items():
            # Sort scores (probably unecessary but just in case)
            # top_hits = sorted(results, key=lambda x: x[0], reverse=True)
            top_hits = results

            relevant_docs_ids = self.ground_truth[query_index]
            # Accuracy@k - We count the result correct, if at least one relevant doc is across the top-k documents
            found_any_acc_k = False
            for k_val in self.accuracy_at_k:
                for hit in top_hits[0:k_val]:
                    if hit[1] in relevant_docs_ids:
                        num_hits_at_k[k_val] += 1
                        found_any_acc_k = True
                        break
            if not found_any_acc_k:
                self.problematic_queries.add(query_index)

            # Precision and Recall@k
            for k_val in self.precision_recall_at_k:
                num_correct = 0
                for hit in top_hits[0:k_val]:
                    if hit[1] in relevant_docs_ids:
                        num_correct += 1
                
                precisions_at_k[k_val].append(num_correct / k_val)
                recall_at_k[k_val].append(num_correct / len(relevant_docs_ids))

            # @Mean Reciprocal Rank
            for k_val in self.mrr_at_k:
                for rank, hit in enumerate(top_hits[0:k_val]):
                    if hit[1] in relevant_docs_ids:
                        MRR[k_val] += 1.0 / (rank + 1)
                        break

            # NDCG@k (normalized discounted cumulative gain at k)
            for k_val in self.ndcg_at_k:
                dcg = 0
                idcg = 0
                for i in range(k_val):
                    if i < len(top_hits):
                        if top_hits[i][1] in relevant_docs_ids:
                            dcg += 1 / np.log2(i + 2)
                    idcg += 1 / np.log2(i + 2)
                ndcg[k_val].append(dcg / idcg)
            
            # Map@k
            for k_val in self.map_at_k:
                num_correct = 0
                sum_precisions = 0
                for i, hit in enumerate(top_hits[0:k_val]):
                    if hit[1] in relevant_docs_ids:
                        num_correct += 1
                        sum_precisions += num_correct / (i + 1)
                avg_precision = sum_precisions / min(k_val, len(relevant_docs_ids))
                AveP_at_k[k_val].append(avg_precision)

            
        # Compute averages
        for k in num_hits_at_k:
            num_hits_at_k[k] /= len(self.query_embeddings)

        for k in precisions_at_k:
            precisions_at_k[k] = np.mean(precisions_at_k[k])
        
        for k in recall_at_k:
            recall_at_k[k] = np.mean(recall_at_k[k])

        for k in MRR:
            MRR[k] /= len(self.query_embeddings)

        for k in ndcg:
            ndcg[k] = np.mean(ndcg[k])
        
        for k in AveP_at_k:
            AveP_at_k[k] = np.mean(AveP_at_k[k])
        
        return {
            "accuracy@k": num_hits_at_k,
            "precision@k": precisions_at_k,
            "recall@k": recall_at_k,
            "ndcg@k": ndcg,
            "mrr@k": MRR,
            "map@k": AveP_at_k,
        }
    
    def output_scores(self, scores):
        """
        Outputs the evaluation metrics.
        Args:
            metrics (Dict[str, Dict[str, float]]): A dictionary mapping metric names to dictionaries mapping metric values to scores.
        """
        for k in scores["accuracy@k"]:
            logger.info("Accuracy@{}: {:.2f}%".format(k, scores["accuracy@k"][k] * 100))

        for k in scores["precision@k"]:
            logger.info("Precision@{}: {:.2f}%".format(k, scores["precision@k"][k] * 100))

        for k in scores["recall@k"]:
            logger.info("Recall@{}: {:.2f}%".format(k, scores["recall@k"][k] * 100))

        for k in scores["mrr@k"]:
            logger.info("MRR@{}: {:.4f} rank from top".format(k, scores["mrr@k"][k]))

        for k in scores["ndcg@k"]:
            logger.info("NDCG@{}: {:.4f}%".format(k, scores["ndcg@k"][k] * 100))

        for k in scores["map@k"]:
            logger.info("MAP@{}: {:.4f} relevant to query".format(k, scores["map@k"][k] * 100))
    


In [11]:
embeddings_path = f"{base_dir}/{subfolder}/embeddings.npy"
query_embeddings_path = f"{base_dir}/evaluation_dataset_{subfolder}/query_embeddings.npy"

corpus_embeddings = np.load(embeddings_path)
query_embeddings = np.load(query_embeddings_path)
# convert to tensor
corpus_embeddings = torch.from_numpy(corpus_embeddings)
query_embeddings = torch.from_numpy(query_embeddings)
#normalize for dot product (could not normalize for cosine similarity)
corpus_embeddings = F.normalize(corpus_embeddings, p=2, dim=1)
query_embeddings = F.normalize(query_embeddings, p=2, dim=1)

In [12]:
ir_evaluator = MailioInformationRetrievalEvaluator(ground_truth, corpus_embeddings, query_embeddings, similarity_functions=[SimilarityFunction.DOT_PRODUCT])
metrics = ir_evaluator.run()
for name, score in metrics.items():
    logger.info(f"----------------- Similarity function: {name} ----------------")
    ir_evaluator.output_scores(score)

    logger.info(f" ->>>>>>>>>>>>>>>>>>> Found {len(ir_evaluator.get_problematic_queries())} queries with no relevant documents in top 10")
    for problem_query_index in ir_evaluator.get_problematic_queries():
        logger.info(df_queries.iloc[problem_query_index]["query"])

----------------- Similarity function: dot ----------------
Accuracy@1: 68.18%
Accuracy@3: 82.95%
Accuracy@5: 92.05%
Accuracy@10: 96.59%
Precision@1: 68.18%
Precision@3: 30.68%
Precision@5: 20.68%
Precision@10: 11.25%
Recall@1: 56.58%
Recall@3: 73.91%
Recall@5: 81.25%
Recall@10: 85.84%
MRR@10: 1.2845 rank from top
NDCG@10: 19.7897%
MAP@100: 69.4015 relevant to query
 ->>>>>>>>>>>>>>>>>>> Found 3 queries with no relevant documents in top 10
"Akontacija DDPO junij 2020"
"Photo ID pickup details for Igor Rendulic"
"Zoom invitation on April 21, 2021"


In [13]:
#| hide

import nbdev; nbdev.nbdev_export()